# SQLQueryChain - Passing instances - Mondial - Foreign Key - GPT 3.5

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
from urllib.parse import quote  
from langchain.callbacks import get_openai_callback

from dotenv import load_dotenv
import os
import sys
import json
import time
load_dotenv()

experiment_path = '..\..'
path = os.path.abspath('')
module_path = os.path.join(path, experiment_path)
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils


# Schema

In [3]:
SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'

FILE_NAME_RESULT = f"results/10_sql_queries_chatgpt_{SCHEMA}_fk_instances.json"

In [4]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

## Conexão

In [ ]:
json_file_path = f"../../datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)

db_connection

### Utilizando o SQLDatabase para pegar todas as informações do database

In [6]:
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables, samples=2)
db.get_table_names()

c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'coordinates'
  self._metadata.reflect(
c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'source'
  self._metadata.reflect(
c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:112: SAWarning: Did not recognize type 'GEOCOORD' of column 'estuary'
  self._metadata.reflect(


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'economy',
 'encompasses',
 'ethnicgroup',
 'geo_desert',
 'geo_estuary',
 'geo_island',
 'geo_lake',
 'geo_mountain',
 'geo_river',
 'geo_sea',
 'geo_source',
 'island',
 'islandin',
 'ismember',
 'lake',
 'lakeonisland',
 'language',
 'located',
 'locatedon',
 'mergeswith',
 'mountain',
 'mountainonisland',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion',
 'river',
 'riveronisland',
 'riverthrough',
 'sea']

In [7]:
len(include_tables)

46

In [ ]:
db.get_foreign_keys()

## Criando o prompt

In [9]:
from langchain.prompts.prompt import PromptTemplate

f = open(f"prompts/prompt_template_sql_query_chain.txt", "r")
prompt_template = f.read()
f.close()


PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"], template=prompt_template
)

print(PROMPT)

input_variables=['input', 'table_info', 'top_k'] output_parser=None partial_variables={} template='You are an Oracle SQL expert. Given an input question, first create a syntactically correct Oracle SQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, don\'t query for at {top_k} most results or any using the FETCH FIRST n ROWS ONLY clause as per Oracle SQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use TRUNC(SYSDATE) function to get the current date, if the question involves "today". \n\nSome hints:\n- Don\

## Criando o Chain para gerar SQL

In [ ]:
query_chain  = create_sql_query_chain(ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k'), db.db, prompt=PROMPT)
query_chain 

## Preparando as consultas em linguagem natural para rodar no LLM

In [11]:
json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']


In [12]:
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

# Rodando as consultas no LLM para gerar SQL

In [12]:
# A cada X consultas, vai ser gerado um delay de 10s para evitar o bloqueio da API.
number_of_queries_to_delay = 25
count = 0
for instance in queries:
    if count == number_of_queries_to_delay:
        count = 0
        time.sleep(10)
    with get_openai_callback() as cb:
        start_time = time.time()
        sql_query = query_chain.invoke({"question":instance["question"]})
        end_time = time.time()
        instance["query_string"] = sql_query
        instance['total_tokens'] = cb.total_tokens
        instance['prompt_tokens'] = cb.prompt_tokens
        instance['completion_tokens'] = cb.completion_tokens
        instance['total_cost'] = cb.total_cost
        instance['time'] = end_time - start_time
        print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
    save_queries(queries)
    count += 1
queries

1 What is the area of Thailand? SELECT area
FROM country
WHERE name = 'Thailand'
2 What are the provinces with an area greater than 10000? SELECT name
FROM province
WHERE area > 10000
3 What are the languages spoken in Poland? SELECT name
FROM language
WHERE country = 'PL'
4 How deep is Lake Kariba? SELECT depth
FROM lake
WHERE name = 'Lake Kariba'
5 What is the total of provinces of Netherlands? SELECT COUNT(*) FROM province WHERE country = 'NL'
6 What is the percentage of religious people are hindu in thailand? SELECT percentage
FROM religion
WHERE country = 'TH' AND name = 'Hindu'
7 List the number of provinces each river flows through. SELECT r.name, COUNT(DISTINCT rt.province) AS num_provinces
FROM river r
JOIN riverthrough rt ON r.name = rt.river
GROUP BY r.name
ORDER BY num_provinces DESC
8 Find all countries that became independent between 8/1/1910 and 8/1/1950. SELECT name
FROM politics
WHERE independence BETWEEN TO_DATE('8/1/1910', 'MM/DD/YYYY') AND TO_DATE('8/1/1950', 'MM/DD

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area\nFROM country\nWHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 7829,
  'prompt_tokens': 7816,
  'completion_tokens': 13,
  'total_cost': 0.0235},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name\nFROM province\nWHERE area > 10000',
  'type': 'simple',
  'total_tokens': 7834,
  'prompt_tokens': 7822,
  'completion_tokens': 12,
  'total_cost': 0.023514},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name\nFROM language\nWHERE country = 'PL'",
  'type': 'medium',
  'total_tokens': 7829,
  'prompt_tokens': 7817,
  'completion_tokens': 12,
  'total_cost': 0.023499},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth\nFROM lake\nWHERE name = 'Lake Kariba'",
  'type': 'simple',
  'total_tokens': 7830,
  'prompt_tokens': 7816,
  'comple

In [13]:
queries=read_queries()
queries


[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area\nFROM country\nWHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 7829,
  'prompt_tokens': 7816,
  'completion_tokens': 13,
  'total_cost': 0.0235},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name\nFROM province\nWHERE area > 10000',
  'type': 'simple',
  'total_tokens': 7834,
  'prompt_tokens': 7822,
  'completion_tokens': 12,
  'total_cost': 0.023514},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name\nFROM language\nWHERE country = 'PL'",
  'type': 'medium',
  'total_tokens': 7829,
  'prompt_tokens': 7817,
  'completion_tokens': 12,
  'total_cost': 0.023499},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth\nFROM lake\nWHERE name = 'Lake Kariba'",
  'type': 'simple',
  'total_tokens': 7830,
  'prompt_tokens': 7816,
  'comple

In [11]:
sql_query_chain_prompt = query_chain.middle[0].template.format(table_info=db.get_table_info(), top_k=0, input="{input}")
print(sql_query_chain_prompt)

c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:307: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


You are an Oracle SQL expert. Given an input question, first create a syntactically correct Oracle SQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 0 results using the FETCH FIRST n ROWS ONLY clause as per Oracle SQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use TRUNC(SYSDATE) function to get the current date, if the question involves "today". 

Some hints:
- Don't use dounle quotes in column name
Example:
`SELECT "column_name" FROM table` should be `SELECT column_name FROM table`
- Don'

#### Fixing queries

In [13]:
to_fix = [40,59,62,72,85,99]
for pos in to_fix:
    instance = queries[pos]
    q = read_queries()
    with get_openai_callback() as cb:
            start_time = time.time()
            sql_query = query_chain.invoke({"question":instance["question"]})
            end_time = time.time()
            instance["query_string"] = sql_query
            instance['total_tokens'] = cb.total_tokens
            instance['prompt_tokens'] = cb.prompt_tokens
            instance['completion_tokens'] = cb.completion_tokens
            instance['total_cost'] = cb.total_cost
            instance['time'] = end_time - start_time
            q[pos] = instance
            print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
            save_queries(q)

c:\Users\rogerrsn\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\utilities\sql_database.py:307: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


41 How many countries that are close to the Mediterranean Sea? SELECT COUNT(*) FROM encompasses WHERE continent = 'Europe' AND percentage = 100 4.202852249145508 0.023536
60  List the names of capital cities which are the base for organizations in alphabetical order SELECT city.name
FROM city
JOIN organization ON city.name = organization.city
ORDER BY city.name ASC 4.919966220855713 0.023574
63 Show the inflation rate of countries that are washed by the Arabian Sea SELECT c.name, e.inflation
FROM country c
JOIN geo_sea gs ON c.code = gs.country
JOIN sea s ON gs.sea = s.name
JOIN economy e ON c.code = e.country
WHERE s.name = 'Arabian Sea' 4.673165798187256 0.023701000000000003
73 What area is the largest continent? SELECT name, area FROM continent ORDER BY area DESC 3.7189509868621826 0.0235
86 What are the 3 airports with the largest name? SELECT name 
FROM airport 
ORDER BY LENGTH(name) DESC 
FETCH FIRST 3 ROWS ONLY 4.280006647109985 0.023552
100 What is the percentage of industries 